In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import learning_curve
from math import sqrt
from numpy.random import randn
import pickle
import sklearn.metrics
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('cardio_train.csv', sep=';')

In [3]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
class PreProcessing:
    def __init__(self, df):
        self.df = df
    
    def label_encoding(self, columns):
        label_encoder = LabelEncoder()
        for column in columns:
            self.df[column]= label_encoder.fit_transform(self.df[column])
            self.df[column].unique()
    
    def clean_outliers(self, columns):
        for column in columns:   
            q1, q3 = self.df[column].quantile([0.25, 0.75])
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            self.df = self.df[(self.df[column] <= upper_bound )]
            self.df = self.df[(self.df[column] >= lower_bound )]
    
    def missing_value(self):
        self.df=self.df.dropna()
        
    def duplicated_data(self):
        self.df = self.df.drop_duplicates()

    def get_df(self):
        return self.df
    
    def set_df(self, df):
        self.df = df
        
    def min_max_scaller(self, X):
        # define min max scaler
        scaler = MinMaxScaler()
        # transform data
        scaled = scaler.fit_transform(X)
        return scaled
    
    def split_dataset(self, X, y, test_size=0.2, random_state=42):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        return X_train, X_test, y_train, y_test
        

In [5]:
pp = PreProcessing(df)

In [6]:
pp.duplicated_data()

In [7]:
pp.clean_outliers(['height', 'weight', 'ap_lo', 'ap_hi', 'age'])

In [8]:
pp.missing_value()

In [9]:
df = pp.get_df()

In [10]:
X = df.drop(columns=['cardio'], axis='column')
y = df['cardio']

In [11]:
wadidaw

NameError: name 'wadidaw' is not defined

## Data Splitting & Normalization

In [12]:
X_train, X_test, y_train, y_test = pp.split_dataset(X, y)

In [13]:
X_train_scaled = pp.min_max_scaller(X_train)
X_test_scaled = pp.min_max_scaller(X_test)

## Neural Networks

In [14]:
import numpy as np

# Define the sigmoid activator; we ask if we want the sigmoid or its derivative
def sigmoid_act(x, der=False):
    import numpy as np
    
    if (der==True) : #derivative of the sigmoid
        f = 1/(1+ np.exp(- x))*(1-1/(1+ np.exp(- x)))
    else : # sigmoid
        f = 1/(1+ np.exp(- x))
    
    return f

# We may employ the Rectifier Linear Unit (ReLU)
def ReLU_act(x, der=False):
    import numpy as np
    
    if (der == True): # the derivative of the ReLU is the Heaviside Theta
        f = np.heaviside(x, 1)
    else :
        f = np.maximum(x, 0)
    
    return f

In [15]:
'''
Artificial Neural Network Class
'''
class DeepNeuralNetwork:
    import numpy as np # linear algebra
    np.random.seed(0)
    
    '''
    Initialize the DNN;
    HiddenLayer vector : will contain the Layers' info
    w, b, phi = (empty) arrays that will contain all the w, b and activation functions for all the Layers
    mu = cost function
    eta = a standard learning rate initialization. It can be modified by the 'set_learning_rate' method
    '''
    def __init__(self, encoder, scaler) :
        self.HiddenLayer = []
        self.w = []
        self.b = []
        self.phi = []
        self.mu = []
        self.eta = 1 #set up the proper Learning Rate!!
        self.encoder = encoder
        self.scaler = scaler
    
    '''
    add method: to add layers to the network
    '''
    def add(self, lay = (4, 'ReLU') ):
        self.HiddenLayer.append(lay)
    
    '''
    FeedForward method: as explained before. 
    '''
    @staticmethod
    def FeedForward(w, b, phi, x):
        return phi(np.dot(w, x) + b)
        
    '''
    BackPropagation algorithm implementing the Gradient Descent 
    '''
    def BackPropagation(self, x, z, Y, w, b, phi):
        self.delta = []
        
        # We initialize ausiliar w and b that are used only inside the backpropagation algorithm once called        
        self.W = []
        self.B = []
        
        # We start computing the LAST error, the one for the OutPut Layer 
        self.delta.append(  (z[len(z)-1] - Y) * phi[len(z)-1](z[len(z)-1], der=True) )
        
        '''Now we BACKpropagate'''
        # We thus compute from next-to-last to first
        for i in range(0, len(z)-1):
            self.delta.append( np.dot( self.delta[i], w[len(z)- 1 - i] ) * phi[len(z)- 2 - i](z[len(z)- 2 - i], der=True) )
        
        # We have the error array ordered from last to first; we flip it to order it from first to last
        temp_delta = []
        for i in reversed(self.delta):
            temp_delta.append(i)
            
        self.delta = np.array(temp_delta, dtype=object)
        
        # Now we define the delta as the error divided by the number of training samples
        self.delta = self.delta/self.X.shape[0] 
        
        '''GRADIENT DESCENT'''
        # We start from the first layer that is special, since it is connected to the Input Layer
        self.W.append( w[0] - self.eta * np.kron(self.delta[0], x).reshape( len(z[0]), x.shape[0] ) )
        self.B.append( b[0] - self.eta * self.delta[0] )
        
        # We now descend for all the other Hidden Layers + OutPut Layer
        for i in range(1, len(z)):
            self.W.append( w[i] - self.eta * np.kron(self.delta[i], z[i-1]).reshape(len(z[i]), len(z[i-1])) )
            self.B.append( b[i] - self.eta * self.delta[i] )
        
        # We return the descended parameters w, b
        np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)
        return np.array(self.W, dtype=object), np.array(self.B, dtype=object)
    
    
    '''
    Fit method: it calls FeedForward and Backpropagation methods
    '''
    def Fit(self, X_train, Y_train, epochs):            
        print('Start fitting...')
        '''
        Input layer
        '''
        self.X = X_train
        self.Y = Y_train
        accurations= []
        recalls = []
        
        '''
        We now initialize the Network by retrieving the Hidden Layers and concatenating them 
        '''
        print('Model recap: \n')
        print('You are fitting an DNN with the following amount of layers: ', len(self.HiddenLayer))
        
        
        for i in range(0, len(self.HiddenLayer)) :
            
            print('Layer ', i+1)
            print('Number of neurons: ', self.HiddenLayer[i][0])
            if i==0:
                # We now try to use the He et al. Initialization from ArXiv:1502.01852
                self.w.append( np.random.randn(self.HiddenLayer[i][0] , self.X.shape[1])/np.sqrt(2/self.X.shape[1]) )
                self.b.append( np.random.randn(self.HiddenLayer[i][0])/np.sqrt(2/self.X.shape[1]))
                # Old initialization
                #self.w.append(2 * np.random.rand(self.HiddenLayer[i][0] , self.X.shape[1]) - 0.5)
                #self.b.append(np.random.rand(self.HiddenLayer[i][0]))
            
                # Initialize the Activation function
                for act in Activation_function.list_act():
                    if self.HiddenLayer[i][1] == act :
                        self.phi.append(Activation_function.get_act(act))
                        print('\tActivation: ', act)
            
            else :
                # We now try to use the He et al. Initialization from ArXiv:1502.01852
                self.w.append( np.random.randn(self.HiddenLayer[i][0] , self.HiddenLayer[i-1][0] )/np.sqrt(2/self.HiddenLayer[i-1][0]))
                self.b.append( np.random.randn(self.HiddenLayer[i][0])/np.sqrt(2/self.HiddenLayer[i-1][0]))
                # Old initialization
                #self.w.append(2*np.random.rand(self.HiddenLayer[i][0] , self.HiddenLayer[i-1][0] ) - 0.5)
                #self.b.append(np.random.rand(self.HiddenLayer[i][0]))
                
                # Initialize the Activation function
                for act in Activation_function.list_act():
                    if self.HiddenLayer[i][1] == act :
                        self.phi.append(Activation_function.get_act(act))
                        print('\tActivation: ', act)
            
        '''
        Now we start the Loop over the training dataset
        '''  
        for epoch in range (0, epochs):
            print("Epoch : ", epoch + 1)
            for I in range(0, self.X.shape[0]): # loop over the training set
                '''
                Now we start the feed forward
                '''  
                self.z = []

                self.z.append( self.FeedForward(self.w[0], self.b[0], self.phi[0], self.X[I]) ) # First layers

                for i in range(1, len(self.HiddenLayer)): #Looping over layers
                    self.z.append( self.FeedForward(self.w[i] , self.b[i], self.phi[i], self.z[i-1] ) )


                '''
                Here we backpropagate
                '''      
                self.w, self.b  = self.BackPropagation(self.X[I], self.z, self.Y.iloc[I], self.w, self.b, self.phi)

                '''
                Compute cost function
                ''' 
                self.mu.append(
                    (1/2) * np.dot(self.z[len(self.z)-1] - self.Y.iloc[I], self.z[len(self.z)-1] - self.Y.iloc[I]) 
                )
                
            prediction = self.get_accuration(X_train)
            prediction[np.isnan(prediction)] = 0
            print("Accuracy : ", accuracy_score(y_train, prediction))
            accurations.append(accuracy_score(y_train, prediction))
            recalls.append(recall_score(y_train, prediction))
            
        return accurations, recalls
            
        print('Fit done. \n')
        

    
    '''
    predict method
    '''
    def predict(self, X_test):
        
        print('Starting predictions...')
        
        self.pred = []
        self.XX = X_test
        
        for I in range(0, self.XX.shape[0]): # loop over the training set
            
            '''
            Now we start the feed forward
            '''  
            self.z = []
            
            self.z.append(self.FeedForward(self.w[0] , self.b[0], self.phi[0], self.XX[I])) #First layer
    
            for i in range(1, len(self.HiddenLayer)) : # loop over the layers
                self.z.append( self.FeedForward(self.w[i] , self.b[i], self.phi[i], self.z[i-1]))
       
            # Append the prediction;
            # We now need a binary classifier; we this apply an Heaviside Theta and we set to 0.5 the threshold
            # if y < 0.5 the output is zero, otherwise is zero
            self.pred.append( np.heaviside(  self.z[-1] - 0.5, 1)[0] ) # NB: self.z[-1]  is the last element of the self.z list
        
        print('Predictions done. \n')

        return np.array(self.pred)
    
    
    def get_accuration(self, X):
        self.pred = []
        self.XX = X
        
        for I in range(0, self.XX.shape[0]): # loop over the training set
            
            '''
            Now we start the feed forward
            '''  
            self.z = []
            
            self.z.append(self.FeedForward(self.w[0] , self.b[0], self.phi[0], self.XX[I])) #First layer
    
            for i in range(1, len(self.HiddenLayer)) : # loop over the layers
                self.z.append( self.FeedForward(self.w[i] , self.b[i], self.phi[i], self.z[i-1]))
       
            # Append the prediction;
            # We now need a binary classifier; we this apply an Heaviside Theta and we set to 0.5 the threshold
            # if y < 0.5 the output is zero, otherwise is zero
            self.pred.append( np.heaviside(  self.z[-1] - 0.5, 1)[0] ) # NB: self.z[-1]  is the last element of the self.z list

        return np.array(self.pred)
        
    def encode_data(self, data, column):
        list_encode =  self.encoder[column].transform([data])
        return list_encode[0]
    
    def scale_data(self, data):
        return self.scaler.transform([data])
    '''
    We need a method to retrieve the accuracy for each training data to follow the learning of the DNN
    '''
    def get_accuracy(self):
        return np.array(self.mu)
    # This is the averaged version
    def get_avg_accuracy(self):
        import math
        self.batch_loss = []
        for i in range(0, 10):
            self.loss_avg = 0
            # To set the batch in 10 element/batch we use math.ceil method
            # int(math.ceil((self.X.shape[0]-10) / 10.0))    - 1
            for m in range(0, (int(math.ceil((self.X.shape[0]-10) / 10.0))   )-1):
                #self.loss_avg += self.mu[60*i+m]/60
                self.loss_avg += self.mu[(int(math.ceil((self.X.shape[0]-10) / 10.0)) )*i + m]/(int(math.ceil((self.X.shape[0]-10) / 10.0)) )
            self.batch_loss.append(self.loss_avg)
        return np.array(self.batch_loss)
    
    '''
    Method to set the learning rate
    '''
    def set_learning_rate(self, et=1):
        self.eta = et
        
        
'''
layers class
'''
class layers :
    '''
    Layer method: used to call standar layers to add. 
    Easily generalizable to more general layers (Pooling and Convolutional layers)
    '''        
    def layer(p=4, activation = 'ReLU'):
        return (p, activation)

'''
Activation functions class
'''
class Activation_function(DeepNeuralNetwork):
    import numpy as np
    
    def __init__(self) :
        super().__init__()
        
    '''
    Define the sigmoid activator; we ask if we want the sigmoid or its derivative
    '''
    def sigmoid_act(x, der=False):
        if (der==True) : #derivative of the sigmoid
            f = 1/(1+ np.exp(- x))*(1-1/(1+ np.exp(- x)))
        else : # sigmoid
            f = 1/(1+ np.exp(- x))
        return f

    '''
    Define the Rectifier Linear Unit (ReLU)
    '''
    def ReLU_act(x, der=False):
        if (der == True): # the derivative of the ReLU is the Heaviside Theta
            f = np.heaviside(x, 1)
        else :
            f = np.maximum(x, 0)
        return f
    
    def list_act():
        return ['sigmoid', 'ReLU']
    
    def get_act(string = 'ReLU'):
        if string == 'ReLU':
            return ReLU_act
        elif string == 'sigmoid':
            return sigmoid_act
        else :
            return sigmoid_act

In [16]:
X_train_scaled.shape

(50001, 12)

In [27]:
dict_acc_train = {}
dict_acc_test = {}
dict_rec_train = {}
dict_rec_test = {}

epoch = [150]
learning_rate = [0.01]
random_state = [42]



for i in epoch:
    for j in learning_rate:
        for k in random_state:                                 
            model = DeepNeuralNetwork(None, None)
            model.add(layers.layer(11, 'ReLU'))
            model.add(layers.layer(11, 'ReLU'))
            model.add(layers.layer(11, 'ReLU'))
            model.add(layers.layer(11, 'ReLU'))
#             model.add(layers.layer(11, 'ReLU'))
            model.add(layers.layer(1, 'sigmoid'))

            model.set_learning_rate(j)

            epoch = i

            strings = 'epoch '+str(i)+', lr '+str(j)+', rs '+str(k)

            dict_acc_train[strings], dict_rec_train[strings] = model.Fit(X_train_scaled, y_train, epoch)
            
            acc_val = model.get_accuracy()
            acc_avg_val = model.get_avg_accuracy()

            predictions_test = model.predict(X_test_scaled)
            dict_acc_test[strings] = accuracy_score(y_test, predictions_test)
            dict_rec_test[strings] = recall_score(y_test, predictions_test)

Start fitting...
Model recap: 

You are fitting an DNN with the following amount of layers:  5
Layer  1
Number of neurons:  11
	Activation:  ReLU
Layer  2
Number of neurons:  11
	Activation:  ReLU
Layer  3
Number of neurons:  11
	Activation:  ReLU
Layer  4
Number of neurons:  11
	Activation:  ReLU
Layer  5
Number of neurons:  1
	Activation:  sigmoid
Epoch :  1
Accuracy :  0.4879702405951881
Epoch :  2
Accuracy :  0.4743905121897562
Epoch :  3
Accuracy :  0.4925101497970041
Epoch :  4
Accuracy :  0.5260094798104038
Epoch :  5
Accuracy :  0.5376492470150597
Epoch :  6
Accuracy :  0.5506289874202516
Epoch :  7
Accuracy :  0.5578288434231315
Epoch :  8
Accuracy :  0.5626487470250595
Epoch :  9
Accuracy :  0.5657286854262915
Epoch :  10
Accuracy :  0.5685686286274274
Epoch :  11
Accuracy :  0.5723285534289314
Epoch :  12
Accuracy :  0.5744085118297634
Epoch :  13
Accuracy :  0.5751484970300594
Epoch :  14
Accuracy :  0.5756884862302754
Epoch :  15


KeyboardInterrupt: 

In [ ]:
dict_acc_train

In [ ]:
dict_rec_train

In [ ]:
dict_acc_test

In [ ]:
dict_rec_test

In [ ]:
wads = 150

In [ ]:
x_axis = list(range(0, wads))

In [ ]:
plt.plot(x_axis, dict_acc_train['epoch '+ str(wads) +', lr 0.01, rs 42'], label = "akurasi")
plt.plot(x_axis, dict_rec_train['epoch '+ str(wads) +', lr 0.01, rs 42'], label = "recall")
plt.xlabel('epoch')
# plt.ylabel('recall')
plt.ylim([0, 1])
plt.legend()
plt.show()